# Calculus

Let ${\mathbf{x}}$ be an $n$-dimensional vector, the following rules are often used when differentiating functions:

- For all $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\triangledown_{x}\mathbf{Ax} = \mathbf{A}^{\top}$
- For all $\mathbf{A} \in \mathbb{R}^{n \times m}$, $\triangledown_{x}\mathbf{x^{\top}A} = \mathbf{A}$
- For all $\mathbf{A} \in \mathbb{R}^{n \times n}$, $\triangledown_{x}\mathbf{x^{\top}Ax} = (\mathbf{A} + \mathbf{A}^{\top})\mathbf{x}$
- $\triangledown_{x}||\mathbf{x}||^2 = \triangledown_{x}\mathbf{x}^{\top}\mathbf{x} = 2\mathbf{x}$

# Automatic Differentiation

## Backward for Non-Scalar Variables

从技术上而言，当 $y$ 不是一个标量而是一个向量(m-dimensional) 对向量 $x$(n-dimensional)求导，求导结果是一个称之为 Jacobian($m \times n$ matrix)的矩阵。

> 在向量求导中，这属于向量对向量求导。

在深度学习（机器学习）中，通常对向量求导的作用是最小化损失函数，其核心是计算在一个批次中每个样本的偏导数之和，因此在 MXNet 框架中，对于向量对向量求导会按照向量求和的方式求导，也就是将向量对向量求导变为标量对向量求导。

下面的例子就说明了 MXNet 的这种处理方法。

In [1]:
from mxnet import autograd, np, npx

In [2]:
npx.set_np()

In [3]:
# vector x
x = np.arange(4)
x

array([0., 1., 2., 3.])

In [7]:
# 向量对向量求导
x.attach_grad()
with autograd.record():
    y = x * x
y.backward()
y, x.grad

(array([0., 1., 4., 9.]), array([0., 2., 4., 6.]))

In [15]:
# 标量对向量求导
u = x.copy()
u.attach_grad()
with autograd.record():
    v = (u * u).sum()  # v is a scalar
v.backward()
v, u.grad, x.grad == u.grad

(array(14.), array([0., 2., 4., 6.]), array([1., 0., 1., 0.]))

## Detaching Computation



In [16]:
with autograd.record():
    y = x * x
    u = y.detach()
    z = u * x
z.backward()
x.grad, u

(array([0., 1., 4., 9.]), array([0., 1., 4., 9.]))

In [17]:
with autograd.record():
    y = x * x
    z = y * x
z.backward()
x.grad, u

(array([ 0.,  3., 12., 27.]), array([0., 1., 4., 9.]))